## Set up a duckweed genotypes x media growth assay

### Pre-requisites to use this script
1. Precise plate and well positions for your machine defined in Plate_positions.py in the 'utils' subdirectory. 
2. All the parameters defined in the cell 'User paramters'. 
3. Sufficient 24-well plates, sterile media and duckweed plants available. 
4. Jubilee machine set up with Media-dispensing syringe tool (50 mL), Duckweed transfer syringe tool and lab automation bedplate. 


In [1]:
#Importing python libraries downloaded from the internet
import random
import pandas as pd
import os
import json
import time

In [2]:
#Importing python libraries from local files. 
import utils.DuckbotExptSetupUtils as exp
import utils.MachineUtils
from utils.MachineUtils import *
import utils.PlatePositionUtils as pp 

In [14]:
#Establish machine communication 
port = '/dev/ttyACM0' #Check options using  - print([port.name for port in serial.tools.list_ports.comports()])
m = MachineCommunication(port)

## 1. Define user parameters

### Labware config 
All should be in arbitrary Jubilee motor units. Find this manually on the machine. They shouldn't need to be updated until you start using a new reservoir. And if that's the case you could create a labware library and then define the media_reservoir variable as one of the objects in your library. 

In [4]:
#X and Y should get the machine to the center of the reservoir. 
media_reservoir = { 'x' : 75, 'y' : 241,}

#Tool positions
media_syringe = 2 #What jubilee tool position did you define this as?
duckweed_syringe = 3

#Volume calculations
dispenses_per_syringe_fill = 20
dispense_mL = 1.5 #In mL
vol_conversion = 3.9 #One mL is 3.9 units. 
dispense_offset = dispense_mL * vol_conversion

#Z-positions for different actions
z_dict = {"zero": 0, #Note that zeros
          "aspirate" : -53, #Measure this for yourself and your labware/toolheads.
          "dispense" : -34
         }

### Define experimental variables, and desired file names and save locations. 

In [3]:

#1. DEFINE EXPERIMENTAL VARIABLES
genotypes = ["Sp7498", "Lm5500", "Lm8627"] # Replace with names for unique duckweed genotypes
media = ["Mock", "50 mM Salt", "150 mM Salt"] # Replace with names for unique media
reps  = 4 # Replace with your desired number of replicates for each duckweed/media combination. 

#2. DEFINE FILE LOCATION AND NAME
expt_setup_parent_dir = os.getcwd() # Default uses current working directory but you can replace with your own choice. 
expt_setup_dir = "TestDriveExpt_1_08032022" # Name of the folder to hold experiment data and metadata including the setup file
expt_setup_filename = "TestDriveExpt_1_08032022.json" #Name for the experiment setup file (Metadata)

expt_setup_file_path = os.path.join(expt_setup_parent_dir, expt_setup_dir)
print(expt_setup_file_path)

if not os.path.exists(expt_setup_file_path):
    os.mkdir(expt_setup_file_path)     

/Users/Orlando/Documents/Github_clones/duckbot/notebooks/TestDriveExpt_1_08032022


## 2. Create dataframe with experiment metadata

In [6]:
# Creates master list of sample info, shuffles and then assigns to plates and wells. 
master_expt_list = []

for g in genotypes:
    for m in media:
        for x in range(reps):
             master_expt_list.append({"genotype": g, "media": m, "condition_replicate": x + 1})


random.shuffle(master_expt_list)
master_expt_list = exp.assign_plates_and_wells(master_expt_list)
expt_dict = {"sample_info" : master_expt_list}

In [7]:
#Save experimental set up file
os.chdir(expt_setup_file_path)
with open(expt_setup_filename, 'w') as f:
    json.dump(expt_dict, f)

In [9]:
#Import from file (in case user wants to make any manual edits to the JSON file after creating it)

# with open(expt_setup_file_name) as datafile:
#     expt_data = json.load(datafile)

# # Turn samples list into a dataframe
# sample_data = expt_data["sample_info"]
# df = pd.DataFrame(sample_data)

## 3. Label 24-well plates and add to machine

#### Label Plates and add plates to machine

In [10]:
num_plates = df.Plate.nunique()
print(num_plates)

print("This experiment requires {} 24-well plate(s)".format(num_plates))
print("----")
lst = list(range(1,num_plates + 1))
for n in lst:
    print("Label a plate with experiment ID or initials and 'plate {}'".format(n))
print("----")
print ("Place the 24-well plate(s) in the jubilee".format(num_plates))
print ("Start at position 1 and fill empty plate slots in order")

2
This experiment requires 2 24-well plate(s)
----
Label a plate with experiment ID or initials and 'plate 1'
Label a plate with experiment ID or initials and 'plate 2'
----
Place the 24-well plate(s) in the jubilee
Start at position 1 and fill empty plate slots in order


## 2. Dispense media
When prompted insert containers of the relevant sterile media into the input slot on the Jubilee. 

In [4]:
#Retrieve absolute positions of wells from a library and then add those coordinatest to the plate set up dataframe
df = pp.add_well_coords_to_df_from_file(expt_setup_file_path, expt_setup_filename)
print(df)

   genotype        media  condition_replicate    plate_well_id    Plate Well  \
0    Lm8627   50 mM Salt                    4  Plate_1_Well_A1  Plate_1   A1   
1    Lm5500  150 mM Salt                    4  Plate_1_Well_A2  Plate_1   A2   
2    Lm5500   50 mM Salt                    1  Plate_1_Well_A3  Plate_1   A3   
3    Lm5500         Mock                    3  Plate_1_Well_A4  Plate_1   A4   
4    Lm8627         Mock                    2  Plate_1_Well_A5  Plate_1   A5   
5    Lm8627  150 mM Salt                    1  Plate_1_Well_A6  Plate_1   A6   
6    Lm8627  150 mM Salt                    4  Plate_1_Well_B1  Plate_1   B1   
7    Sp7498   50 mM Salt                    4  Plate_1_Well_B2  Plate_1   B2   
8    Sp7498         Mock                    3  Plate_1_Well_B3  Plate_1   B3   
9    Lm5500  150 mM Salt                    1  Plate_1_Well_B4  Plate_1   B4   
10   Lm5500         Mock                    1  Plate_1_Well_B5  Plate_1   B5   
11   Lm5500   50 mM Salt                

In [6]:
#Reorganizes dataframe to create machine instructions sorted by media-type
media_dicts = pp.pull_list_of_well_coord_dicts_by_dfcolumn(df, 'media')
print(media_dicts)

[{'media': '150 mM Salt', 'well-coords': [[48.0, 175.0], [124.0, 175.0], [29.0, 175.0], [86.0, 156.0], [48.0, 137.0], [105.0, 137.0], [48.0, 118.0], [29.0, 78.0], [48.0, 78.0], [86.0, 78.0], [29.0, 78.0], [86.0, 59.0]]}, {'media': '50 mM Salt', 'well-coords': [[29.0, 175.0], [67.0, 175.0], [48.0, 156.0], [124.0, 156.0], [124.0, 137.0], [29.0, 118.0], [67.0, 118.0], [124.0, 118.0], [124.0, 78.0], [48.0, 59.0], [105.0, 59.0], [124.0, 59.0]]}, {'media': 'Mock', 'well-coords': [[86.0, 175.0], [105.0, 175.0], [67.0, 156.0], [105.0, 156.0], [29.0, 137.0], [67.0, 137.0], [86.0, 137.0], [86.0, 118.0], [105.0, 118.0], [67.0, 78.0], [105.0, 78.0], [67.0, 59.0]]}]


In [15]:
#Pick up syringe toolhead
m.toolChange(media_syringe)

In [16]:
# Send machine instructions
for media in media_dicts:
    m.moveTo(x=0,y=0,z=0)
    print(f"Please ensure {media['media']} is available in the machine before continuing.")
    print("Change syringe and/or needle if desired")
    while True:
        value = input("Enter 'YES' to confirm that the correct media is in position")
        if value != "YES":
            print("Please confirm")
        else:
            break
    exp.dispense_to_wells(m, media["well-coords"], dispense_offset, dispenses_per_syringe_fill, media_reservoir, z_dict)

Please ensure 150 mM Salt is available in the machine before continuing.
Change syringe and/or needle if desired


Enter 'YES' to confirm that the correct media is in position YES


Move to Z = zero
Move to reservoir position
moved to height for aspiration
G1    E70.20 F1000.00
Moved to Z = zero
Hovering over the first well to dispense into
Prepare to dispense
X = 48.0
Y = 175.0
G1    E-5.85 F1000.00
Prepare to dispense
X = 124.0
Y = 175.0
G1    E-5.85 F1000.00
Prepare to dispense
X = 29.0
Y = 175.0
G1    E-5.85 F1000.00
Prepare to dispense
X = 86.0
Y = 156.0
G1    E-5.85 F1000.00
Prepare to dispense
X = 48.0
Y = 137.0
G1    E-5.85 F1000.00
Prepare to dispense
X = 105.0
Y = 137.0
G1    E-5.85 F1000.00
Prepare to dispense
X = 48.0
Y = 118.0
G1    E-5.85 F1000.00
Prepare to dispense
X = 29.0
Y = 78.0
G1    E-5.85 F1000.00
Prepare to dispense
X = 48.0
Y = 78.0
G1    E-5.85 F1000.00
Prepare to dispense
X = 86.0
Y = 78.0
G1    E-5.85 F1000.00
Prepare to dispense
X = 29.0
Y = 78.0
G1    E-5.85 F1000.00
Prepare to dispense
X = 86.0
Y = 59.0
G1    E-5.85 F1000.00
Please ensure 50 mM Salt is available in the machine before continuing.
Change syringe and/or needle if desire

Enter 'YES' to confirm that the correct media is in position YES


Move to Z = zero
Move to reservoir position
moved to height for aspiration
G1    E70.20 F1000.00
Moved to Z = zero
Hovering over the first well to dispense into
Prepare to dispense
X = 29.0
Y = 175.0
G1    E-5.85 F1000.00
Prepare to dispense
X = 67.0
Y = 175.0
G1    E-5.85 F1000.00
Prepare to dispense
X = 48.0
Y = 156.0
G1    E-5.85 F1000.00
Prepare to dispense
X = 124.0
Y = 156.0
G1    E-5.85 F1000.00
Prepare to dispense
X = 124.0
Y = 137.0
G1    E-5.85 F1000.00
Prepare to dispense
X = 29.0
Y = 118.0
G1    E-5.85 F1000.00
Prepare to dispense
X = 67.0
Y = 118.0
G1    E-5.85 F1000.00
Prepare to dispense
X = 124.0
Y = 118.0
G1    E-5.85 F1000.00
Prepare to dispense
X = 124.0
Y = 78.0
G1    E-5.85 F1000.00
Prepare to dispense
X = 48.0
Y = 59.0
G1    E-5.85 F1000.00
Prepare to dispense
X = 105.0
Y = 59.0
G1    E-5.85 F1000.00
Prepare to dispense
X = 124.0
Y = 59.0
G1    E-5.85 F1000.00
Please ensure Mock is available in the machine before continuing.
Change syringe and/or needle if desired

Enter 'YES' to confirm that the correct media is in position YES


Move to Z = zero
Move to reservoir position
moved to height for aspiration
G1    E70.20 F1000.00
Moved to Z = zero
Hovering over the first well to dispense into
Prepare to dispense
X = 86.0
Y = 175.0
G1    E-5.85 F1000.00
Prepare to dispense
X = 105.0
Y = 175.0
G1    E-5.85 F1000.00
Prepare to dispense
X = 67.0
Y = 156.0
G1    E-5.85 F1000.00
Prepare to dispense
X = 105.0
Y = 156.0
G1    E-5.85 F1000.00
Prepare to dispense
X = 29.0
Y = 137.0
G1    E-5.85 F1000.00
Prepare to dispense
X = 67.0
Y = 137.0
G1    E-5.85 F1000.00
Prepare to dispense
X = 86.0
Y = 137.0
G1    E-5.85 F1000.00
Prepare to dispense
X = 86.0
Y = 118.0
G1    E-5.85 F1000.00
Prepare to dispense
X = 105.0
Y = 118.0
G1    E-5.85 F1000.00
Prepare to dispense
X = 67.0
Y = 78.0
G1    E-5.85 F1000.00
Prepare to dispense
X = 105.0
Y = 78.0
G1    E-5.85 F1000.00
Prepare to dispense
X = 67.0
Y = 59.0
G1    E-5.85 F1000.00


## 5. Transfer duckweed
Place a container filled with fronds of the relevant duckweed type and the machine will attempt to move individual fronds into the relevant wells. After each attempt at filling all wells a camera will take pictures of each well to confirm success and then unsuccessful wells will be reattempted

In [ ]:
## testing duckweed transfer
pp.fetch_well_position(2, 'A1')

In [ ]:
# manually probe the z height of the duckweed reservoir surface
# enter the z value here
duckweed_reservoir = [75, 241, -64.45]
m.moveTo(x=duckweed_reservoir[0], y=duckweed_reservoir[1])

In [ ]:
# aspirate duckweed
# M203 E2500 to increase max speed
# M201 E2000 to increase max acc
def aspirate_duckweed(reservoir):
    # move to a random point to pick up duckweed
    # assumes dense petri dish
    r = 25
    rx = r * random.random()
    ry = r * random.random()

    # move in xy first
    m.moveTo(x=reservoir[0] + rx, y=reservoir[1] + ry)
    m.move(de=10, s=1000) # prime the syringe. hopefully not necessary with 10cc.
    # aspirate duckweed
    # first move to surface
    m.moveTo(z=reservoir[2])
    m.dwell(3000)
    m.move(dz=-0.75) # press slightly
    m.dwell(3000)
    m.move(dz=8, de=40, s=2000) # aspirate!
    

In [ ]:
# test aspirate
aspirate_duckweed(duckweed_reservoir)
m.dwell(5000)
m.move(de=-20, s=1500)
#m.move(dz=10, de=-25, s=2100) # aspirate!

In [ ]:
#Automated transfer
grouped_df = df.groupby('genotype')
for field_value, sample_df in grouped_df:
    print("Place container of duckweed type **{0}** into jubilee and ensure lid is open".format(field_value))
    print("""Type anything into the input field to confirm that the media is available.
    After this point the Jubilee will begin dispensing""")
    input() 
    count = 0
    for index,s in sample_df.iterrows():
        #move to plate and well
        aspirate_duckweed(duckweed_reservoir)
        m.moveTo(z=-42)
        well = pp.fetch_well_position(s["Plate"][-1], str(s["Well"]))
        m.moveTo(x=well['x'], y=well['y'])
        m.move(de=-25, s=2000) # dispense duckweed 
        count += 1
        if count > 3:
            break
        #print("Move to {0}, well {1}".format(s["Plate"], s["Well"]))
        #print("Dispensing media of type {0} into {1}, well {2}".format(field_value,s["Plate"], s["Well"]))


In [ ]:
#Manual transfer

grouped_df = df.groupby('genotype')
for field_value, sample_df in grouped_df:
    print("Place container of duckweed type **{0}** into jubilee and ensure lid is open".format(field_value))
    for index,s in sample_df.iterrows():
        plate = s["Plate"]
        well = s["Well"]
        print(f"Transfer {field_value} to plate {plate}, well {well}")
    input()
        #print("Dispensing media of type {0} into {1}, well {2}".format(field_value,s["Plate"], s["Well"]))


Place container of duckweed type **Lm5500** into jubilee and ensure lid is open
Transfer Lm5500 to plate Plate_1, well A2
Transfer Lm5500 to plate Plate_1, well A3
Transfer Lm5500 to plate Plate_1, well A4
Transfer Lm5500 to plate Plate_1, well B4
Transfer Lm5500 to plate Plate_1, well B5
Transfer Lm5500 to plate Plate_1, well B6
Transfer Lm5500 to plate Plate_1, well C2
Transfer Lm5500 to plate Plate_1, well C3
Transfer Lm5500 to plate Plate_1, well C4
Transfer Lm5500 to plate Plate_1, well D3
Transfer Lm5500 to plate Plate_1, well D6
Transfer Lm5500 to plate Plate_2, well A4


Place container of duckweed type **Lm8627** into jubilee and ensure lid is open
Transfer Lm8627 to plate Plate_1, well A1
Transfer Lm8627 to plate Plate_1, well A5
Transfer Lm8627 to plate Plate_1, well A6
Transfer Lm8627 to plate Plate_1, well B1
Transfer Lm8627 to plate Plate_1, well C6
Transfer Lm8627 to plate Plate_1, well D1
Transfer Lm8627 to plate Plate_2, well A2
Transfer Lm8627 to plate Plate_2, well A3
Transfer Lm8627 to plate Plate_2, well A5
Transfer Lm8627 to plate Plate_2, well B3
Transfer Lm8627 to plate Plate_2, well B4
Transfer Lm8627 to plate Plate_2, well B5


Place container of duckweed type **Sp7498** into jubilee and ensure lid is open
Transfer Sp7498 to plate Plate_1, well B2
Transfer Sp7498 to plate Plate_1, well B3
Transfer Sp7498 to plate Plate_1, well C1
Transfer Sp7498 to plate Plate_1, well C5
Transfer Sp7498 to plate Plate_1, well D2
Transfer Sp7498 to plate Plate_1, well D4
Transfer Sp7498 to plate Plate_1, well D5
Transfer Sp7498 to plate Plate_2, well A1
Transfer Sp7498 to plate Plate_2, well A6
Transfer Sp7498 to plate Plate_2, well B1
Transfer Sp7498 to plate Plate_2, well B2
Transfer Sp7498 to plate Plate_2, well B6


In [ ]:
m.move(de=10, s=800)

In [18]:
m.moveTo(z=z_dict["zero"])